# VI. Time series: visualizing

---
**Author(s):** Quentin Yeche, Kenji Ose, Dino Ienco - [UMR TETIS](https://umr-tetis.fr) / [INRAE](https://www.inrae.fr/)

---

## 1. Introduction

In this notebook we will introduce some tips for visualizing and masking time series.

## 2. Import libraries

As usual, we import all the required Python libraries.

In [1]:
# STAC access
import pystac_client
import planetary_computer

# dataframes
import pandas as pd

# xarrays
import xarray as xr

# library for turning STAC objects into xarrays
import stackstac

# visualization
from matplotlib import pyplot as plt

# miscellanous
import numpy as np
from IPython.display import display
from datetime import date

## 3. Reloading of Sentinel-2 time series

Here we run the code of the [previous notebook](Joensuu_05-Time-series_part01.ipynb) in order to get the Sentinel-2 images time series, in the region of Montpellier.

In [ ]:
aoi_bounds = (3.875107329166124, 43.48641456618909, 4.118824575734205, 43.71739887308995)
cloud_nb = 20
# retrieving the relevant STAC Item
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
    )

today = date.today()
last_month = today.replace(month=today.month-1).strftime('%Y-%m')
time_range = f"2020-01-01/2021-01-01"#{last_month}"
search = catalog.search(
    collections=['sentinel-2-l2a'],
    datetime=time_range,
    bbox=[3.944092,43.526638,4.014816,43.568420],#aoi_bounds,
    query={"sat:relative_orbit": {"eq": 8}, 
           "eo:cloud_cover": {"lt": cloud_nb}},
    sortby="datetime"
)
items = search.item_collection()
print(f"{len(items)} items found")

time_steps_pc = len(items)

bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B11', 'B12', 'SCL']
FILL_VALUE = 2**16-1
array = stackstac.stack(
                    items,
                    assets = bands,
                    resolution=10,
                    dtype="uint16",
                    fill_value=FILL_VALUE,
                    bounds_latlon=aoi_bounds,
                    chunksize= (time_steps_pc, 1, 'auto', 'auto')
                    )
array.drop_duplicates('time')
array 

## 4. Example of ploting time series images

### 4.1. Plotting by default

Plot of the first 6 dates of data xarray in natural color composite.

In [ ]:
source = array.sel(band=["B08", "B04", "B03"])
rgb = source[:6]
rgb.plot.imshow(col_wrap=3, col="time", rgb="band", vmax=2500, size=4)

### 4.2. Cloud masking

For information, SCL band is coded as follows:

|Bit| value Class             |
|---|-------------------------|
|0  | No data                 |
|1  | Saturated or defective  |
|2  | Dark area pixels        |
|3  | Cloud shadows           |
|4  | Vegetation              |
|5  | Bare Soil               |
|6  | Water                   |
|7  | Unclassified            |
|8  | Cloud medium probability|
|9  | Cloud high probability  |
|10 | Thin cirrus             |
|11 | Snow or ice             |
   
Values related to cloud and cloud shadows are kept here.

In [ ]:
# mask creation
SCL = array.sel(band = 'SCL')
mask = SCL.isin([3, 8, 9, 10])

# application of mask
result = array.where(~mask) #, 0) argument 2 = new output value for nan

source = result.sel(band=["B08", "B04", "B03"])
rgb = source[:6]
rgb.plot.imshow(col_wrap=3, col="time", rgb="band", vmax=2500, size=4)

## Estimation of time interval between S2 scenes

In [ ]:
#result.time.diff("time").dt.days.plot.hist()

In [ ]:
#type(result)
#test = result.drop_duplicates('time')
#toto = np.array(result.time)
#values, counts = np.unique(toto, return_counts=True)
#idx = np.where(counts>1)[0]
#print(values[idx])
#print(counts[idx])

interpolated = source.interpolate_na(dim="time", method="linear", use_coordinate = 'time')
interpolated = interpolated.ffill(dim= 'time')
interpolated.data = interpolated.data.astype(np.uint16)
interpolated.plot.imshow(col_wrap=3, col="time", rgb="band", vmax=2500, size=4)
#print('Band: {} - Interpolation complete'.format(band))

#print(len(test))

#newlist = [] # empty list to hold unique elements from the list
#duplist = [] # empty list to hold the duplicate elements from the list
#for i in mylist:
#    if i not in newlist:
#        newlist.append(i)
#    else:
#        duplist.append(i) # this method catches the first duplicate entries, and appends them to the list
# The next step is to print the duplicate entries, and the unique entries
#print("List of duplicates", duplist)
#print("Unique Item List", newlist) # prints the final list of unique items
#test2 = test.interpolate_na(dim="time", method="linear")


## Conversion of S2 time series into median time series

Reduce time dimension in function of quartile (4 images a year) and time with median.

In [ ]:
composites = result.resample(time="Q").median("time")
composites

Selection of three bands for rgb output

In [ ]:
rgb = composites.sel(band=["B04", "B03", "B02"])

## Final cleanup to make a nicer-looking animation

- Forward-fill any NaN pixels from the previous frame, to make the animation look less jumpy.
- Also skip the first frame, since its NaNs can’t be filled from anywhere.


In [ ]:
cleaned = rgb.ffill("time")[1:]

## Production of animated GIF

Use GeoGIF to turn the stack into an animation. 

In [ ]:
gif_img = geogif.dgif(cleaned, fps=8).compute()
gif_img